In [2]:
import torch
import torch.nn as nn
from torchvision import models

In [7]:
class SE(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SE, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [3]:
model = models.resnet18(pretrained=True)

In [5]:
newmodel = nn.Sequential(*list(model.children())[:-2])

In [8]:
img = torch.rand(1, 3, 224, 224)
res = newmodel(img)
res.shape

torch.Size([1, 512, 7, 7])

In [9]:
m1 = SE(512)
res1 = m1(res)
res1.shape

torch.Size([1, 512, 7, 7])

In [10]:
m2 = nn.AdaptiveAvgPool2d((1,1))
res2 = m2(res1)
res2.shape

torch.Size([1, 512, 1, 1])

In [11]:
res2 = res2.squeeze()

In [12]:
res2.shape

torch.Size([512])